In [109]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text
bs = BeautifulSoup(source, 'xml')
table=bs.find('table')
sol = pd.DataFrame(columns = ['Postalcode','Borough','Neighborhood'])
for x in table.find_all('tr'):
    row_data=[]
    for y in x.find_all('td'):
        row_data.append(y.text.strip())
    if len(row_data)==3:
        sol.loc[len(sol)] = row_data
sol=sol[sol['Borough']!='Not assigned']
sol = sol.groupby(['Postalcode','Borough'], sort=False).agg(', '.join)
sol.reset_index(inplace=True)
sol['Neighborhood'] = np.where(sol['Neighborhood'] == 'Not assigned',sol['Borough'], sol['Neighborhood'])
sol.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [108]:
sol.shape

(103, 3)

In [111]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [120]:
import geocoder
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude
data=pd.read_csv('http://cocl.us/Geospatial_data')

In [121]:
data.rename(columns={'Postal Code':'Postalcode'},inplace=True)
data_combined = pd.merge(data, sol, on='Postalcode')
final_data=data_combined[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
final_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
